Southern University of Science and Technology-Department of Computer Science and Engineering

Course: Machine Learning(CS 405)-Professor: Qi Hao

## Homework #2
#### Due date: October, 7th, 2019

In [2]:
"""
Import libraries that you might require.

"""

import numpy as np
import math
import matplotlib.pyplot as plt
import operator

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

We will implement the KNN algorithm for the breast cancer dataset. Refer to the pdf and the following functions for the instructions. Complete all the functions as indicated below. The four functions would be autograded as mentioned in the pdf.

In [3]:
"""
Task 1: Classification

Please implement KNN for K: 3, 5, and 7 with the following norms:
L1
L2
L-inf
"""

# Read data (Breast Cancer Dataset). Remember to comment out the code not contained in a function.
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()

X = breast['data']
y = breast['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:400], y[:400]
X_val, y_val = X[400:500], y[400:500]
X_test, y_test = X[500:], y[500:]


def distanceFunc(metric_type, x_sample, x_train):
    """
    Computes the distance between two d-dimension vectors. 
    
    Please DO NOT use Numpy's norm function when implementing this function. 
    
    Args:
        metric_type (str): Metric: L1, L2, or L-inf
        vec1 ((d,) np.ndarray): d-dim vector
        vec2 ((d,)) np.ndarray): d-dim vector
    
    Returns:
        distance (float): distance between the two vectors
    """

    # diff = vec1 - vec2
    # if metric_type == "L1":
    #     distance = np.sum(np.abs(diff)) #complete

    # if metric_type == "L2":
    #     distance = np.sqrt(np.sum(diff**2)) #complete
        
    # if metric_type == "L-inf":
    #     distance = np.max(np.abs(diff)) #complete

    sample = np.array(x_sample)
    X_train = np.array(x_train)
    
    sample = np.repeat(sample, X_train.shape[0], axis=0)
    train = np.tile(X_train, (x_sample.shape[0], 1))


    if metric_type == "L1":
        delta = sample-train
        L1 = np.sum(np.abs(delta), axis=1, keepdims=True)
        L1 = L1.reshape(x_sample.shape[0] ,X_train.shape[0])
        distance = L1

    if metric_type == "L2":
        L2 = np.sqrt(np.sum(delta**2, axis=1, keepdims=True))
        L2 = L2.reshape(x_sample.shape[0] ,X_train.shape[0])
        distance = L2

    if metric_type == "L-inf":
        L3 = np.max(np.abs(delta), axis=1, keepdims=True)
        L3 = L3.reshape(x_sample.shape[0] ,X_train.shape[0])
        distance = L3
        
    # construction of distance looks like
    # [L11, L12, L13, ..., L1n] --> sample_1, L with 400 X_trains
    # [L21, L22, L23, ..., L2n] --> sample_2, L with 400 X_trains
    # [...]
    # [Lm1, Lm2, Lm3, ..., Lmn] --> sample_m, L with 400 X_trains
    
    return distance


def computeDistancesNeighbors(K, metric_type, X_train, y_train, sample):
    """
    Compute the distances between every datapoint in the train_data and the 
    given sample. Then, find the k-nearest neighbors.
    
    Return a numpy array of the label of the k-nearest neighbors.
    
    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        sample ((p,) np.ndarray): Single sample whose distance is to computed with every entry in the dataset
        
    Returns:
        neighbors (list): K-nearest neighbors' labels
    """

    # You will also call the function "distanceFunc" here
    # Complete this function

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    sample = np.array(sample)

    distance = distanceFunc(metric_type, sample, X_train)

    neighbors_index = np.argpartition(distance, K, axis=1)[:, :K]
    neighbors_index = np.sort(neighbors_index, axis=1)
    
    neighbors = y_train[neighbors_index]
    
    # The construction of neighbors looks like:
    # in case that K = 5
    # [1,0,1,1,1] --> nearst 5 points's lable of sample_1
    # [0,0,1,1,1] --> nearst 5 points's lable of sample_2

    return neighbors


def Majority(neighbors):
    """
    Performs majority voting and returns the predicted value for the test sample.
    
    Since we're performing binary classification the possible values are [0,1].
    
    Args:
        neighbors (list): K-nearest neighbors' labels
        
    Returns:
        predicted_value (int): predicted label for the given sample
    """
    
    # Performs majority voting
    # Complete this function

    count_0 = np.sum(neighbors == 0, axis=1)
    count_1 = np.sum(neighbors == 1, axis=1)
    predicted_value = np.zeros([neighbors.shape[0],1])
    # predicted_value = np.zeros(neighbors.shape[0])

    # Perform majority voting
    for idex in range(0, neighbors.shape[0]):
        if count_0[idex] > count_1[idex]:
            predicted_value[idex] = 0
        else:
            predicted_value[idex] = 1
    
    return predicted_value


def KNN(K, metric_type, X_train, y_train, X_val):
    """
    Returns the predicted values for the entire validation or test set.
    
    Please DO NOT use Scikit's KNN model when implementing this function. 

    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        X_val ((n, p) np.ndarray): Validation or test data
        
    Returns:
        predicted_values (list): output for every entry in validation/test dataset 
    """
    
    # Complete this function
    # Loop through the val_data or the test_data (as required)
    # and compute the output for every entry in that dataset  
    # You will also call the function "Majority" here

    neighbors = computeDistancesNeighbors(K, metric_type, X_train, y_train, X_val)
    predictions = Majority(neighbors)

    return predictions


def evaluation(predicted_values, actual_values):
    """
    Computes the accuracy of the given datapoints.
    
    Args:
        predicted_values ((n,) np.ndarray): Predicted values for n samples
        actual_values ((n,) np.ndarray): Actual values for n samples
    
    Returns:
        accuracy (float): accuracy
    """
    
    return accuracy_score(predicted_values, actual_values)


def main():
    """
    Calls the above functions in order to implement the KNN algorithm.
    
    Test over the following range K = 3,5,7 and all three metrics. 
    In total you will have nine combinations to try.
    
    PRINTS out the accuracies for the nine combinations on the validation set,
    and the accuracy on the test set for the selected K value and appropriate norm.
    
    REMEMBER: You have to report these values by populating the Table 1.
    """
    
    ## Complete this function
    
    K = [3,5,7]
    norm = ["L1", "L2", "L-inf"]
    
    print("<<<<VALIDATION DATA PREDICTIONS>>>>")
    ## Complete
    
    print("<<<<TEST DATA PREDICTIONS>>>>")
    
    ## Complete


In [8]:
pre = KNN(3, 'L1', X_train, y_train, X_val)
print(pre)

[[0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [5]:
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()

X = breast['data']
y = breast['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:400], y[:400]
X_val, y_val = X[400:500], y[400:500]
X_test, y_test = X[500:], y[500:]

print(np.array(X_train).shape)
print(np.array(X_val).shape)
print(np.array(X_test).shape)

print(np.array(y_train).shape)
print(np.array(y_val).shape)
print(np.array(y_test).shape)
print(np.array(y_train[1]))

(400, 30)
(100, 30)
(69, 30)
(400,)
(100,)
(69,)
1


In [6]:
# L1，L2，L-inf矩阵计算思路

import numpy as np

X_train = np.array(X_train)
X_test = np.array(X_val)
test = np.repeat(X_test, X_train.shape[0], axis=0)
train = np.tile(X_train, (X_test.shape[0], 1))

delta = test-train
L1 = np.sum(np.abs(delta), axis=1, keepdims=True)
L1 = L1.reshape(X_test.shape[0] ,X_train.shape[0])
print(L1.shape)

L2 = np.sqrt(np.sum(delta**2, axis=1, keepdims=True)) #complete
L2 = L2.reshape(X_test.shape[0] ,X_train.shape[0])
print(L2.shape)

L3 = np.max(np.abs(delta), axis=1, keepdims=True) #complete
L3 = L3.reshape(X_test.shape[0] ,X_train.shape[0])
print(L3.shape)

(100, 400)
(100, 400)
(100, 400)


Uncomment the code below to run the main function (Remember to recomment the code before submitting).

In [7]:
# Finally, call the main function
# main()

Answer the following questions here:

1. How could having a larger dataset influence the performance of KNN?

2. Tabulate your results from `main()` in the table provided.

3. Finally, mention the best K and the norm combination you have settled upon and report the accuracy on the test set using that combination.